In [1]:
# import correct packages

import pandas as pd
import json
import time
import statistics
import matplotlib
import numpy as np

In [2]:
# read the json file

df = pd.read_json('data.json',lines=True)

In [3]:
# initialize dictionaries that will track year and price


#bed_bath keys/categories
#bed1_bath1
#bed1_bath15
#bed2_bath1
#bed2_bath15
#bed2_bath2
#bed2_bath25
#bed3_bath2
#bed3_bath25




#bedroom4 = {}
#bedroom5 = {}


#the large dictionary
#this splits the data by year, and then by room type, and makes it searchable by zpid
#the data will be stored at the lowest level as zpid:avg_cost (over the year)
#we are assuming that there are no more than 2 price updates per year and that they are equally weighted
#OR that price updates are not significantly larger


# data = {2010:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},
#             bed3_bath2:{}, bed3_bath25:{}},
#         2011:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},
#             bed2_bath2:{},bed2_bath25:{},bed3_bath2:{}, bed3_bath25:{}},
#         2012:{bed1_bath1:{},bed1_bath15:{},
#             bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},bed3_bath2:{}, bed3_bath25:{}},
#         2013:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},
#               bed3_bath2:{}, bed3_bath25:{}},
#         2014:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},
#               bed3_bath2:{}, bed3_bath25:{}},
#         2015:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},
#               bed3_bath2:{}, bed3_bath25:{}},
#         2016:{bed1_bath1:{},bed1_bath15:{},bed2_bath1:{},bed2_bath15:{},bed2_bath2:{},bed2_bath25:{},
#               bed3_bath2:{}, bed3_bath25:{}}}


data = {2009:{}, 2010:{}, 2011:{}, 2012:{}, 2013:{}, 2014:{}, 2015:{}, 2016:{}}



c = 0

In [4]:
#what I want to do
#for now, we will do this with no bed_bath breakdowns

#iterate through all of the data points by zpid (because that's the format that we have)
#for each data point, get the year
#CHECK IF YEAR IS WITHIN RANGE
#call up the subdictionary of that year
                                                                    #get the bed_bath values
                                                                    #CHECK IF BED_BATH IS WITHIN RANGE
                                                                    #call up the subdictionary of that bed_bath
#get the zpid (we have this already)
#search the subdictionary for that zpid
    #if zpid found, average existing value and new value of cost
    #else, add zpid:cost

#THEN
#go through each dictionary year for each bed_bath type
#if zpid in current year and not in following year
    #append zpid:previous average to following year

#THEN
#using a try-catch block, make lists of housing price changes

In [5]:
# separate data by year and zpid
# the key is the year from [-4:]
c = 0
count = 0
for zpid in df['zpid'].tolist():
    homeValue = df.loc[df['zpid'] == zpid]['data'][c]['HomeValueChartData']
    chart = json.loads(homeValue)
    for i in chart['points']:
        year = int(time.ctime(i['x']/1000)[-4:])
        if year in data:
            if zpid in data[year]:
                data[year][zpid] = (i['y'] + data[year][zpid]) * 0.5
                count = count+1
            else:
                data[year][zpid] = (i['y'])
    c = c+1
print(count)

1199120


In [6]:
# add any missing values from a following year
for year in [2010,2011,2012,2013,2014,2015]:
    for zpid in data[year]:
        if zpid not in data[year+1]:
            data[year+1][zpid] = data[year][zpid]
#            print("added data point from ", year)

In [7]:
# initialize difference lists
diff_data = {"10_11":[], "11_12":[], "12_13":[], "13_14":[], "14_15":[], "15_16":[]}
for zpid in data[2015]:
    diff_data["15_16"].append((data[2016][zpid] - data[2015][zpid]) / data[2015][zpid])
for zpid in data[2014]:
    diff_data["14_15"].append((data[2015][zpid] - data[2014][zpid]) / data[2014][zpid])
for zpid in data[2013]:
    diff_data["13_14"].append((data[2014][zpid] - data[2013][zpid]) / data[2013][zpid])
for zpid in data[2012]:
    diff_data["12_13"].append((data[2013][zpid] - data[2012][zpid]) / data[2012][zpid])
for zpid in data[2011]:
    diff_data["11_12"].append((data[2012][zpid] - data[2011][zpid]) / data[2011][zpid])
for zpid in data[2010]:
    diff_data["10_11"].append((data[2011][zpid] - data[2010][zpid]) / data[2010][zpid])

In [12]:
for key in diff_data:
    print(len(diff_data[key]))
    print(str(np.mean(diff_data[key])*100) + "%")
    print(str(np.std(diff_data[key])*100) + "%")
    print(' ')

15870
-1.92750120824%
7.72994379027%
 
15870
4.81149409595%
11.8075901919%
 
15870
6.97962084036%
12.4318866857%
 
15870
4.79556550722%
8.43546629985%
 
15870
0.380393977054%
6.38393663319%
 
15870
6.681767177%
12.0545541496%
 
